In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
import argparse
import tqdm 
import time 

In [2]:

env.workspace = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MS.gdb'
# contour_fc = 'Elev_Contours'
# elevation_field = 'ContourElevations'
# output_dem = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MSdem_fromCreateDEM.tif'

In [3]:
def create_dem(contour_fc, elevation_field, output_dem): 
    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("Spatial")
    if not arcpy.Exists(contour_fc):
        raise Exception(f"fc not found")
    field_names = [f.name for f in arcpy.ListFields(contour_fc)]
    if elevation_field not in field_names:
        raise Exception(f"'{elevation_field}' was not found in {contour_fc}. Available fields are: {field_names}")
    try: 
        dem = TopoToRaster([[contour_fc, elevation_field, 'Contour']])
    except Exception as e: 
        raise Exception(f"Error with TopoToRaster: {str(e)}")
    try: 
        dem.save(output_dem)
    except Exception as e: 
        raise Exception(f"Error saving DEM: {str(e)}")

In [ ]:
create_dem(contour_fc='Elev_Contours', elevation_field='ContourElevation', output_dem='C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MSdem_fromCreateDEM.tif')

In [4]:
env.workspace = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MS.gdb'
env.overwriteOutput = True

In [5]:
contours = 'Elev_Contours'
boundary = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/Boundary/MS_StateBoundary_2015.shp'
clipped_contours = 'clipped_contours'
output_DEM = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/dem.tif'

In [6]:
arcpy.analysis.Clip(contours, boundary, clipped_contours)

<Result 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MS.gdb/clipped_contours'>

In [7]:
fields = [f.name for f in arcpy.ListFields('Elev_Contours')]
print(fields)

['OBJECTID', 'Shape', 'ContourElevation', 'ContourUnits', 'FCODE', 'ContourInterval', 'Depression', 'IndexFlag', 'CELL_ID', 'ProcessDate', 'MapID', 'NEDResolution', 'NEDDate', 'Shape_Length']


In [9]:
contour_fc = 'Elev_Contours'
fields = arcpy.ListFields(contour_fc)

In [10]:
print(f'checking feature class: {contour_fc}')
print('available fields:')
for field in fields: 
    print(f" - {field.name} ({field.type})")

checking feature class: Elev_Contours
available fields:
 - OBJECTID (OID)
 - Shape (Geometry)
 - ContourElevation (Double)
 - ContourUnits (Integer)
 - FCODE (Integer)
 - ContourInterval (Double)
 - Depression (SmallInteger)
 - IndexFlag (SmallInteger)
 - CELL_ID (Integer)
 - ProcessDate (Date)
 - MapID (String)
 - NEDResolution (SmallInteger)
 - NEDDate (Integer)
 - Shape_Length (Double)


In [11]:
if not arcpy.Exists('Elev_Contours'):
    print("ElevContorus not found in workspace")

In [ ]:
dem = TopoToRaster([
    ["Elev_Contours", "ContourElevation", "Contour"]
    ])
# arcpy.env.mask = boundary
# dem.save(output_DEM)
# print("DEM created successfully")

In [34]:
# Load MS USGS Surface Contours 

env.workspace = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MS.gdb'
feature_classes = arcpy.ListFeatureClasses()
for fc in feature_classes:
    print(fc)

in_features ='C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/Boundary/MS_StateBoundary_2015.shp'
out_location = 'C:/Users/JamieGood/OneDrive - Vaulted/Documents/VSCode/Data/MS.gdb'
out_name = "stateBoundary"
arcpy.FeatureClassToFeatureClass_conversion(in_features, out_location, out_name)

Elev_Contours


<Result 'C:\\Users\\JamieGood\\OneDrive - Vaulted\\Documents\\VSCode\\Data\\MS.gdb\\stateBoundary'>

In [35]:
with arcpy.da.SearchCursor("Elev_Contours", ["SHAPE@", "FieldName"]) as cursor: 
    for row in cursor: 
        geometry = row[0]
        attribute_value = row[1]
        print(f"SHape: {geometry.type}, Attribute: {attribute_value}")

RuntimeError: Cannot find field 'FieldName'

In [ ]:
# Set params for TopoToRaster

TopoBoundary = ([state_boundary])
TopoContour = ([surface_contours,'Elev_Contours'])